# Тестирование скорости обработки данных эндпоинтами Azure OpenAI

Этот ноутбук предназначен для измерения скорости обработки данных различными эндпоинтами Azure OpenAI.

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import json
from tqdm.notebook import tqdm
import numpy as np

# Константы для моделей
GPT_3_5_TURBO = "gpt-3.5-turbo"
GPT_4_TURBO_PREVIEW = "gpt-4-turbo-preview"
GPT_4 = 'gpt-4'
GPT_4o = 'gpt-4o'

In [ ]:
# Загрузка переменных окружения
_ = load_dotenv(find_dotenv())

# Список эндпоинтов для тестирования
ENDPOINTS = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_ENDPOINT_4o-3",
    "AZURE_OPENAI_ENDPOINT_4o-1",
    "AZURE_OPENAI_ENDPOINT_4o-1-mini"
]

# Количество запросов для каждого эндпоинта
NUM_REQUESTS = 5

# Размер батча для тестирования
BATCH_SIZE = 3

In [ ]:
def create_azure_client(endpoint: str) -> AzureOpenAI:
    """Создает клиент Azure OpenAI для указанного эндпоинта."""
    return AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-07-01-preview",
        azure_endpoint=endpoint
    )

def get_model_name(endpoint: str) -> str:
    """Извлекает имя модели из имени эндпоинта."""
    # Извлекаем имя модели из URL эндпоинта
    endpoint_name = endpoint.split('/')[-1]
    
    # Маппинг имен эндпоинтов на имена моделей
    model_mapping = {
        "4o-3": "4o-3",
        "4o-1": "4o-1",
        "4o-1-mini": "4o-1-mini"
    }
    
    return model_mapping.get(endpoint_name, GPT_4o)

def create_test_batch() -> Tuple[List[Dict], List[Dict]]:
    """Создает тестовый батч карточек для обработки."""
    source_cards = []
    secondary_cards = []
    
    for i in range(BATCH_SIZE):
        source_cards.append({
            "word": f"test_word_{i}",
            "sentence": f"This is a test sentence {i}.",
            "id": f"test_{i}"
        })
        
        secondary_cards.append({
            "word": f"тестовое_слово_{i}",
            "sentence": f"Это тестовое предложение {i}.",
            "id": f"test_{i}"
        })
    
    return source_cards, secondary_cards

def test_endpoint(endpoint: str, num_requests: int) -> List[float]:
    """Тестирует скорость обработки данных эндпоинтом."""
    client = create_azure_client(endpoint)
    model = get_model_name(endpoint)
    response_times = []
    
    for _ in tqdm(range(num_requests), desc=f"Testing {model}"):
        source_cards, secondary_cards = create_test_batch()
        
        start_time = time.time()
        
        # Генерация переводов
        response = client.chat.completions.create(
            model=model,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a multilingual assistant."},
                {"role": "user", "content": json.dumps({
                    "source_cards": source_cards,
                    "secondary_cards": secondary_cards
                })}
            ]
        )
        
        end_time = time.time()
        response_times.append(end_time - start_time)
        
    return response_times

In [ ]:
# Запуск тестов
results = {}

for endpoint_env in ENDPOINTS:
    endpoint = os.getenv(endpoint_env)
    if endpoint:
        model = get_model_name(endpoint)
        print(f"\nТестирование эндпоинта: {endpoint}")
        print(f"Модель: {model}")
        results[model] = test_endpoint(endpoint, NUM_REQUESTS)

In [ ]:
# Анализ результатов
df = pd.DataFrame(results)

# Вычисление статистик
stats = pd.DataFrame({
    'mean': df.mean(),
    'std': df.std(),
    'min': df.min(),
    'max': df.max(),
    'median': df.median()
})

print("Статистика времени ответа (в секундах):")
display(stats)

In [ ]:
# Визуализация результатов
plt.figure(figsize=(12, 6))

# Box plot
plt.subplot(1, 2, 1)
sns.boxplot(data=df)
plt.title('Распределение времени ответа')
plt.xticks(rotation=45)
plt.ylabel('Время (секунды)')

# Bar plot средних значений
plt.subplot(1, 2, 2)
sns.barplot(x=stats.index, y='mean', data=stats)
plt.title('Среднее время ответа')
plt.xticks(rotation=45)
plt.ylabel('Время (секунды)')

plt.tight_layout()
plt.show()

In [ ]:
# Сохранение результатов
timestamp = time.strftime("%Y%m%d-%H%M%S")
results_file = f"endpoint_speed_test_results_{timestamp}.json"

results_data = {
    "timestamp": timestamp,
    "num_requests": NUM_REQUESTS,
    "batch_size": BATCH_SIZE,
    "raw_results": results,
    "statistics": stats.to_dict()
}

with open(results_file, 'w') as f:
    json.dump(results_data, f, indent=2)

print(f"Результаты сохранены в файл: {results_file}")